# el_landing_to_bronze_adventureworks

In [1]:
import pyspark
from pyspark.sql import SparkSession
import logging
from datetime import datetime
from pyspark.sql.functions import lit
from configs import configs
from functions import functions as F

from dotenv import load_dotenv
import os


## Import Environment

In [2]:
load_dotenv()

HOST_ADDRESS=os.getenv('HOST_ADDRESS')
MINIO_ACCESS_KEY=os.getenv('MINIO_ACCESS_KEY')
MINIO_SECRET_KEY=os.getenv('MINIO_SECRET_KEY')

## Spark Session

In [3]:
spark = SparkSession.builder \
        .appName("el_landing_to_bronze_adventureworks") \
        .config("spark.hadoop.fs.s3a.endpoint", f"http://{HOST_ADDRESS}:9000") \
        .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS_KEY) \
        .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET_KEY) \
        .config("spark.hadoop.fs.s3a.path.style.access", True) \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
        .config("hive.metastore.uris", "thrift://metastore:9083") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .getOrCreate()

## Log configs

In [4]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

logging.info("Starting convertions from Minio to Minio Delta Adventureworks...")

2024-08-21 10:12:47,107 - INFO - Starting convertions from Minio to Minio Delta Adventureworks...


## Path configs

In [5]:
input_prefix_layer_name = configs.prefix_layer_name['0']
table_input_name = configs.lake_path['landing_adventure_works']
output_prefix_layer_name = configs.prefix_layer_name['1']
storage_output = configs.lake_path['bronze']

## Process

In [6]:
for key, value in configs.tables_postgres_adventureworks.items():
    table = value
    #print(value)
    table_name = F.convert_table_name(table)
    #print(table_name)
    
    try:
        df_input_data = spark.read.format("parquet").load(f'{table_input_name}{input_prefix_layer_name}{table_name}')
        df_with_update_date = F.add_metadata(df_input_data)
        df_with_update_date.write.format("delta").mode("overwrite").partitionBy('month_key').save(f'{storage_output}{output_prefix_layer_name}{table_name}')
        logging.info(f'Table {table_name} sucessfully processed and saved to Minio: {storage_output}{output_prefix_layer_name}{table_name}')   
    except Exception as e:
        logging.error(f'Error processing table {table}: {str(e)}')
        
logging.info("Convertion from parquet to Delta completed was sucessfully!")

2024-08-21 10:12:56,046 - INFO - Table sales_countryregioncurrency sucessfully processed and saved to Minio: s3a://bronze/adventure_works/bronze_sales_countryregioncurrency
2024-08-21 10:12:58,389 - INFO - Table sales_creditcard sucessfully processed and saved to Minio: s3a://bronze/adventure_works/bronze_sales_creditcard
2024-08-21 10:12:59,568 - INFO - Table sales_currency sucessfully processed and saved to Minio: s3a://bronze/adventure_works/bronze_sales_currency
2024-08-21 10:13:00,704 - INFO - Table humanresources_department sucessfully processed and saved to Minio: s3a://bronze/adventure_works/bronze_humanresources_department
2024-08-21 10:13:01,774 - INFO - Table humanresources_employee sucessfully processed and saved to Minio: s3a://bronze/adventure_works/bronze_humanresources_employee
2024-08-21 10:13:03,515 - INFO - Table sales_salesorderheader sucessfully processed and saved to Minio: s3a://bronze/adventure_works/bronze_sales_salesorderheader
2024-08-21 10:13:03,516 - INFO -